<a href="https://colab.research.google.com/github/alibelhrak/python_chatbot_scratch/blob/main/Cha_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk


In [ ]:
import nltk
nltk.download('wordnet')


# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout   , Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import categorical_crossentropy
import nltk
from nltk.stem import WordNetLemmatizer


# Preparing intentes

In [ ]:
intents_data={
  "intents": [
    {
      "tag": "greeting",
      "patterns": ["Hello", "Hi", "Hey", "What's up", "Good morning"],
      "responses": ["Hello!", "Good day!", "Hey there!", "Hi! How can I assist you?"]
    },
    {
      "tag": "goodbye",
      "patterns": ["Bye", "Goodbye", "See you later", "Take care"],
      "responses": ["Goodbye!", "See you soon!", "Take care!", "Have a great day!"]
    },
    {
      "tag": "thanks",
      "patterns": ["Thank you", "Thanks", "Appreciate it"],
      "responses": ["You're welcome!", "Glad I could help!", "Anytime!"]
    },
    {
      "tag": "help",
      "patterns": ["Can you help me?", "I need assistance", "Help me"],
      "responses": ["Sure! What do you need help with?", "I'm here to assist!", "Tell me what you need help with."]
    },
    {
      "tag": "weather",
      "patterns": ["What's the weather like?", "Is it raining today?", "Tell me the forecast"],
      "responses": ["I can check the weather for you!", "Do you want today's or this week's forecast?", "Let me get that info for you."]
    },
    {
      "tag": "joke",
      "patterns": ["Tell me a joke", "Make me laugh", "Do you know any jokes?"],
      "responses": ["Why don’t skeletons fight each other? They don’t have the guts!", "Why did the scarecrow win an award? Because he was outstanding in his field!"]
    }
  ]
}


In [3]:
intents_data_json = pd.read_json('/content/intents.json')

In [4]:
intents_data_json

,intents
0,"{'tag': 'greeting', 'patterns': ['Hi', 'How ar..."
1,"{'tag': 'goodbye', 'patterns': ['Bye', 'See yo..."
2,"{'tag': 'thanks', 'patterns': ['Thanks', 'Than..."
3,"{'tag': 'isn't', 'patterns': ['this isn't work..."
4,"{'tag': 'hours', 'patterns': ['What hours are ..."
...,...
715,"{'tag': 'anger disorder', 'patterns': ['i cons..."
716,"{'tag': 'anxiety anger', 'patterns': ['i const..."
717,"{'tag': 'abruptly', 'patterns': ['hello i'm 20..."
718,"{'tag': 'anger outbursts', 'patterns': ['hello..."


In [5]:
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [7]:
lemmatizer = WordNetLemmatizer()
words=[]
classes=[]
documents=[]
ignore_symbols = ['.','?' , ';' , ':' , '!' , '%']
from nltk.tokenize import word_tokenize

for intent in intents_data_json["intents"]:
    for pattern in intent["patterns"]:
        words_list = word_tokenize(pattern)
        words.append(words_list)
        documents.append((words_list, intent["tag"]))

        if intent["tag"] not in classes:
            classes.append(intent["tag"])

print("Documents:", documents)

Documents: [(['Hi'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['this', 'is', "n't", 'working'], "isn't"), (['What', 'hours', 'are', 'you', 'open', '?'], 'hours'), (['What', 'are', 'your', 'hours', '?'], 'hours'), (['When', 'are', 'you', 'open', '?'], 'hours'), (['What', 'kind', 'of', 'treatment', 'do', 'you', 'provide', '?'], 'treatments'), (['What', 'kinds', 'of', 'treatments', 'are', 'there', '?'], 'treatments'), (['What', 'do', 'you', 'provide', '?'], 'treatments'), (['Is', 'this', 'covered', 'by', 'OHIP', '?'], 'payments'), (['Do', 'I', 'have', 'to', 'pay', 'for', 'this', '?'], 'payments'), (['How', 'can', 'I', 'pay', '?'], 'payments'), (['Are', 'you', 'open', 'today', '?'], 'opentoday'), (['When

In [9]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [10]:
words = [lemmatizer.lemmatize(word) for sublist in words for word in sublist if word not in ignore_symbols]
words = list(sorted(set(words)))
classes = list(sorted(set(classes)))


# saving files

In [11]:
import pickle

with open('words.pkl' , 'wb') as f:
  pickle.dump(words , f)

with open('classes.pkl' , 'wb') as f:
  pickle.dump(classes , f)

# Trianing Data

In [12]:
import random
training = []
output_empty = [0] * len(classes)

for document in documents:
  bag=[]
  word_patterns = document[0]
  word_patterns =[lemmatizer.lemmatize(word) for word in word_patterns]
  for word in words:
    bag.append(1) if word in word_patterns else bag.append(0)
  outputrow = list(output_empty)
  outputrow[classes.index(document[1])] = 1
  training.append([bag  , outputrow])

random.shuffle(training)
training = np.array(training, dtype=object)
train_x = list(training[: , 0])
train_y = list(training[: , 1])

# Building the model

In [13]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(len(train_x[0]),)))
model.add(Dropout(0.25))
model.add(Dense(64 , activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(len(train_y[0]) , activation = 'softmax'))

sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(optimizer = sgd ,  metrics=['accuracy'] , loss='categorical_crossentropy' )

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
model.fit(np.array(train_x) , np.array(train_y) , epochs=200 , batch_size =5 , verbose=0)

# Building ChatBot

In [31]:
import random
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Assuming lemmatizer, words, classes, and model are already defined
lemmatizer = WordNetLemmatizer()
ignore_symbols = ['?', '!', '.', ',', "'s", "'m"]

def clean_up(sentence):
    sentence_words = word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words if word not in ignore_symbols]
    return sentence_words

def bag_of_words(sentence):
    sentence_words = clean_up(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    predicted_word = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.1  # Adjusted threshold

    results = [[i, r] for i, r in enumerate(predicted_word) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)

    print(f"DEBUG: Results after threshold: {results}")  # Debugging print

    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})

    print(f"DEBUG: Return list: {return_list}")  # Debugging print

    return return_list

def get_responses(intent_list, intent_json):
    tag = intent_list[0]['intent']
    list_of_intents = intent_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

# Assuming intents_data is already loaded as a dictionary
while True:
    message = input("Vous: ")

    if message.lower() in ['quit', 'exit', 'goodbye']:
        print("Chatbot: Thanks for this conversation! Goodbye 😊")
        break

    ints = predict_class(message)

    if not ints:
        print("Chatbot: Sorry, this message contains things beyond my knowledge!")
    else:
        res = get_responses(ints, intents_data)
        print("Chatbot:", res)

Vous: hi 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
DEBUG: Results after threshold: []
DEBUG: Return list: []
Chatbot: Sorry, this message contains things beyond my knowledge!


KeyboardInterrupt: Interrupted by user